# testscript.ipynb

A Jupyter notebook for testing and development of the AmBIENCe2ABM module.

In [ ]:
## Import modules

import pandas as pd
import numpy as np
from itertools import product
import ambience2abm as amb

In [ ]:
## Read the raw data and assumptions.

ambience = amb.AmBIENCeDataset(
    interior_node_depth=0.1,
    period_of_variations=2225140,
)
ambience.data

In [ ]:
## Create unique building periods

building_periods = ambience.building_periods()
building_periods

In [ ]:
## Check the building type to stock mappings.

ambience.building_type_mappings

In [ ]:
## Check the building stock statistics

ambience.building_stock_statistics

In [ ]:
## Check structure type assumptions

ambience.structure_types

In [ ]:
## Structure statistics

ss = pd.DataFrame(
    [
        [
            r["REFERENCE BUILDING USE CODE"],
            r["building_period"],
            r["REFERENCE BUILDING COUNTRY CODE"],
            st,
            r["material_combination_weight"]
            * r[" ".join(
                [
                    "REFERENCE BUILDING",
                    ambience.structure_types.loc[st,"mapping"],
                    "U-VALUE (W/m2/K)"
                ]
            )],
            r["material_combination_weight"]
            * ambience.calculate_weighted_effective_thermal_mass(
                r, st,
            ),
            r["material_combination_weight"]
            * ambience.structure_types.loc[st,"linear_thermal_bridge_W_mK"]
        ] + [
            r["material_combination_weight"] * x
            for x in ambience.calculate_U_values(r,st)
        ]
        for ((i,r),st) in product(
            ambience.data.iterrows(),
            ambience.structure_types.index
        )
    ],
    columns=[
        "building_type",
        "building_period",
        "location_id",
        "structure_type",
        "design_U_value_m2K",
        "effective_thermal_mass_J_m2K",
        "linear_thermal_bridges_W_mK",
        "external_U_value_to_ambient_air_W_m2K",
        "external_U_value_to_ground_W_m2K",
        "internal_U_value_to_structure_W_m2K",
        "total_U_value_W_m2K",
    ]
).groupby([
    "building_type",
    "building_period",
    "location_id",
    "structure_type"
]).agg({
    "design_U_value_m2K": "sum",
    "effective_thermal_mass_J_m2K": "sum",
    "linear_thermal_bridges_W_mK": "sum",
    "external_U_value_to_ambient_air_W_m2K": "sum",
    "external_U_value_to_ground_W_m2K": "sum",
    "internal_U_value_to_structure_W_m2K": "sum",
    "total_U_value_W_m2K": "sum",
})
ss